In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import tensorflow as tf

import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.keras.compat import keras

In [3]:
x, y = make_classification(n_samples=1000, n_features=10, random_state=42, n_classes=2)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.20, random_state=42)

In [10]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(10,)),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

**Model Pruning**

In [11]:
pruning_params = {
    'pruning_schedule' : tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.5,
        begin_step=0,
        end_step=1000
    )
}

model = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

In [14]:
# Train with pruning
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_dense   (None, 128)               2690      
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_dense_  (None, 64)                16450     
 1 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dense_  (None, 1)                 131       
 2 (PruneLowMagnitude)                                           
                                                                 
Total params: 19271 (75.29 KB)
Trainable params: 9729 (38.00 KB)
Non-trainable params: 9542 (37.29 KB)
_________________________________________________________________


In [15]:
model.fit(x_train, y_train, epochs=10, batch_size=32, callbacks=[tfmot.sparsity.keras.UpdatePruningStep()])

Epoch 1/10
25/25 [==============================] - 1s 1ms/step - loss: 0.5220 - accuracy: 0.7962
Epoch 2/10
25/25 [==============================] - 0s 855us/step - loss: 0.3404 - accuracy: 0.8712
Epoch 3/10
25/25 [==============================] - 0s 855us/step - loss: 0.3074 - accuracy: 0.8750
Epoch 4/10
25/25 [==============================] - 0s 1ms/step - loss: 0.2951 - accuracy: 0.8763
Epoch 5/10
25/25 [==============================] - 0s 896us/step - loss: 0.2875 - accuracy: 0.8800
Epoch 6/10
25/25 [==============================] - 0s 959us/step - loss: 0.2792 - accuracy: 0.8838
Epoch 7/10
25/25 [==============================] - 0s 958us/step - loss: 0.2743 - accuracy: 0.8850
Epoch 8/10
25/25 [==============================] - 0s 959us/step - loss: 0.2691 - accuracy: 0.8850
Epoch 9/10
25/25 [==============================] - 0s 875us/step - loss: 0.2650 - accuracy: 0.8850
Epoch 10/10
25/25 [==============================] - 0s 833us/step - loss: 0.2590 - accuracy: 0.8888


In [16]:
# Strip pruning wrapper
model = tfmot.sparsity.keras.strip_pruning(model)

**Quantization**

- Post-training quantization

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp_8vvlo6k\assets


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp_8vvlo6k\assets


- Quantization-aware training

In [18]:
qat_model = tfmot.quantization.keras.quantize_model(model)
qat_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

qat_model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
25/25 [==============================] - 1s 958us/step - loss: 0.2592 - accuracy: 0.8875
Epoch 2/10
25/25 [==============================] - 0s 1000us/step - loss: 0.2476 - accuracy: 0.8950
Epoch 3/10
25/25 [==============================] - 0s 938us/step - loss: 0.2435 - accuracy: 0.9000
Epoch 4/10
25/25 [==============================] - 0s 917us/step - loss: 0.2389 - accuracy: 0.8950
Epoch 5/10
25/25 [==============================] - 0s 917us/step - loss: 0.2333 - accuracy: 0.9013
Epoch 6/10
25/25 [==============================] - 0s 896us/step - loss: 0.2273 - accuracy: 0.9013
Epoch 7/10
25/25 [==============================] - 0s 959us/step - loss: 0.2219 - accuracy: 0.9038
Epoch 8/10
25/25 [==============================] - 0s 938us/step - loss: 0.2187 - accuracy: 0.9150
Epoch 9/10
25/25 [==============================] - 0s 896us/step - loss: 0.2173 - accuracy: 0.9062
Epoch 10/10
25/25 [==============================] - 0s 917us/step - loss: 0.2073 - accuracy: 0.916